In [6]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('no_training_all_question_answers.csv')
df

,SEASON,CLUE VALUE,CLUE CATEGORY,QUESTION,OUR MODEL_ANSWER,REAL ANSWER,CORRECT,CONTEXT USED
0,34,200,REJECTED GREETING CARDS,"\""Have fun in"" this country! ""But remember the...",against the advice of the U.S. Department of S...,Afghanistan,False,"Global Relief Foundation . In November 2001, d..."
1,34,400,REJECTED GREETING CARDS,"\""It's OK, applicant! You didn't want to go to...",University of Houston,Stanford,False,Lauro Cruz . Born in Beaumont to Manuel Cruz a...
2,34,600,REJECTED GREETING CARDS,"\""Tummyache? Next time, take a close look at"" ...",NaN,clams,False,No Context
3,34,800,REJECTED GREETING CARDS,"\""Nice surgery! You finally vented"" this soft,...",The spleen,spleen,True,Splenic injury . The spleen is an organ in the...
4,34,1000,REJECTED GREETING CARDS,"\""Sorry you two had"" these 14-letter ""differen...","""irreconcilable differences"",",irreconcilable,True,"Tony Parker . On 17 November 2010, Longoria fi..."
...,...,...,...,...,...,...,...,...
16310,16,100,COUNTRIES BY COLLEGE,"The University of Sunderland, the University o...",University of Northumbria at Newcastle.,England,False,Stephen Newton (artist) . Newton was born in G...
16311,16,200,COUNTRIES BY COLLEGE,"The Emile Cohl School, the University of Toulo...",NaN,France,False,No Context
16312,16,300,COUNTRIES BY COLLEGE,"Chiba University, Waseda University, Fukuoka J...",Tokai University,Japan,False,Tokai University Fukuoka Junior College . Toka...
16313,16,200,HIRSCHFELD,Broadway's Diamond Lil; come up & see her some...,Diamond Lil (play,Mae West,False,Diamond Lil (play) . Diamond Lil is a 1928 pla...


In [3]:
# General stats

# s34 aired September 11, 2017
# s16 aired September 6, 1999
seasons = df['SEASON'].value_counts()
print(f'season # --> # of questions in that season\n{seasons}\n')

categories = df['CLUE CATEGORY'].value_counts()
print(f'category name --> # of questions in that category\n{categories}\n')

# total questions answered correctly/incorrectly
percent_answer_correct = df['CORRECT'].value_counts(normalize=True)
print(f'percent answered incorrectly and correctly\n{percent_answer_correct}\n')

season # --> # of questions in that season
34    13362
16     2953
Name: SEASON, dtype: int64

category name --> # of questions in that category
AMERICAN HISTORY          36
WORLD HISTORY             30
BEFORE & AFTER            25
BOOKS & AUTHORS           25
ISLANDS                   25
                          ..
CURRENT POLITICIANS        1
FIRST LADY FACTS           1
AMERICAN BUSINESS          1
U.S. POLITICAL HISTORY     1
ROMAN HISTORY              1
Name: CLUE CATEGORY, Length: 3338, dtype: int64

percent answered incorrectly and correctly
False    0.822924
True     0.177076
Name: CORRECT, dtype: float64



In [4]:
# Investigate answers that were empty strings
nan_rows = df[df['OUR MODEL_ANSWER'].isna()]
print(nan_rows)

percent_nan_rows = df['OUR MODEL_ANSWER'].isna().mean() * 100
print(f'percentage of answers that were empty strings aka Nan: {percent_nan_rows}')

# Investigate no context
percentage_with_no_context = (df['CONTEXT USED'] == 'No Context').mean() * 100
print(f"percentage of rows with 'no context' is: {percentage_with_no_context}")

       SEASON  CLUE VALUE             CLUE CATEGORY  \
2          34         600   REJECTED GREETING CARDS   
9          34         400           SELF-HELP BOOKS   
14         34         400             BACK IN BLACK   
16         34         800             BACK IN BLACK   
20         34         600           HIGHWAY TO HELL   
...       ...         ...                       ...   
16300      16        1000             \"B" YOURSELF   
16307      16         100  NEW YORK CITY TELEVISION   
16308      16         300  NEW YORK CITY TELEVISION   
16311      16         200      COUNTRIES BY COLLEGE   
16314      16         300                HIRSCHFELD   

                                                QUESTION OUR MODEL_ANSWER  \
2      \"Tummyache? Next time, take a close look at" ...              NaN   
9      In dealing with others, we need to set these l...              NaN   
14     On the verge of going bust in 1997, it was sav...              NaN   
16     It took more than a "mom

In [5]:
# Investigate question length

average_words = df['QUESTION'].str.split().apply(len).mean()
average_words_true = df[df['CORRECT'] == True]['QUESTION'].str.split().apply(len).mean()
average_words_false = df[df['CORRECT'] == False]['QUESTION'].str.split().apply(len).mean()

print(f'The average number of words in a question is: {average_words:.3f}')
print(f'The average number of words in a question for CORRECT=True is: {average_words_true:.3f}')
print(f'The average number of words in a question for CORRECT=False is: {average_words_false:.3f}')

The average number of words in a question is: 14.951
The average number of words in a question for CORRECT=True is: 15.489
The average number of words in a question for CORRECT=False is: 14.835


In [39]:
# Investigate named entities

# spacy built in entity types
# PERSON - People, including fictional.
# NORP - Nationalities or religious or political groups.
# FAC - Buildings, airports, highways, bridges, etc.
# ORG - Companies, agencies, institutions, etc.
# GPE - Countries, cities, states.
# LOC - Non-GPE locations, mountain ranges, bodies of water.
# PRODUCT - Objects, vehicles, foods, etc. (Not services.)
# EVENT - Named hurricanes, battles, wars, sports events, etc.
# WORK_OF_ART - Titles of books, songs, etc.

import spacy

nlp = spacy.load('en_core_web_lg')      # need to run python -m spacy download en_core_web_lg

def extract_named_entities(text):
    doc = nlp(str(text))
    entities = [ent.text for ent in doc.ents if ent.label_ == 'PERSON' or 
                                                ent.label_ == 'NORP' or
                                                ent.label_ == 'FAC' or
                                                ent.label_ == 'ORG' or
                                                ent.label_ == 'GPE' or
                                                ent.label_ == 'LOC' or
                                                ent.label_ == 'PRODUCT' or
                                                ent.label_ == 'EVENT' or
                                                ent.label_ == 'WORK_OF_ART']
    return entities

df['named_entities'] = df['REAL ANSWER'].apply(extract_named_entities)

percentage_with_named_entities = (df['named_entities'].apply(lambda x: bool(x))).mean() * 100

correct_rows = df[df['CORRECT'] == True]
incorrect_rows = df[df['CORRECT'] == False]

percentage_correct_with_named_entities = (correct_rows['named_entities'].apply(lambda x: bool(x))).mean() * 100
percentage_incorrect_with_named_entities = (incorrect_rows['named_entities'].apply(lambda x: bool(x))).mean() * 100

print(f"The percentage of rows with a named entity in 'REAL ANSWER': {percentage_with_named_entities:.3f}%")
print(f"The percentage of rows with a named entity in 'REAL ANSWER' where CORRECT=True: {percentage_correct_with_named_entities:.3f}%")
print(f"The percentage of rows with a named entity in 'REAL ANSWER' where CORRECT=False: {percentage_incorrect_with_named_entities:.3f}%")

percentage_without_named_entities = (~df['named_entities'].astype(bool)).mean() * 100

# percentage_correct_without_named_entities = (correct_rows['named_entities'].apply(lambda x: not bool(x))).mean() * 100
# percentage_incorrect_without_named_entities = (incorrect_rows['named_entities'].apply(lambda x: not bool(x))).mean() * 100

percentage_correct_without_named_entities = (~correct_rows['named_entities'].astype(bool)).mean() * 100
percentage_incorrect_without_named_entities = (~incorrect_rows['named_entities'].astype(bool)).mean() * 100

print(f"The percentage of rows without a named entity: {percentage_without_named_entities:.3f}%")
print(f"The percentage of correct rows without a named entity: {percentage_correct_without_named_entities:.3f}%")
print(f"The percentage of incorrect rows without a named entity: {percentage_incorrect_without_named_entities:.3f}%")

KeyboardInterrupt: 

In [ ]:
def extract_people(text):
    doc = nlp(str(text))
    entities = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    return entities

df['PERSON'] = df['REAL ANSWER'].apply(extract_people)

percentage_with_named_entities = (df['PERSON'].apply(lambda x: bool(x))).mean() * 100

correct_rows = df[df['CORRECT'] == True]
incorrect_rows = df[df['CORRECT'] == False]

percentage_correct_with_named_entities = (correct_rows['PERSON'].apply(lambda x: bool(x))).mean() * 100
percentage_incorrect_with_named_entities = (incorrect_rows['PERSON'].apply(lambda x: bool(x))).mean() * 100

print(f"The percentage of rows with a PERSON named entity in 'REAL ANSWER': {percentage_with_named_entities:.3f}%")
print(f"The percentage of rows with a PERSON named entity in 'REAL ANSWER' where CORRECT=True: {percentage_correct_with_named_entities:.3f}%")
print(f"The percentage of rows with a PERSON named entity in 'REAL ANSWER' where CORRECT=False: {percentage_incorrect_with_named_entities:.3f}%")


NameError: name 'nlp' is not defined

In [ ]:
def extract_places(text):
    doc = nlp(str(text))
    entities = [ent.text for ent in doc.ents if ent.label_ == 'GPE' or
                                                ent.label_ == 'LOC']
    return entities

df['PLACES'] = df['REAL ANSWER'].apply(extract_places)

percentage_with_named_entities = (df['PLACES'].apply(lambda x: bool(x))).mean() * 100

correct_rows = df[df['CORRECT'] == True]
incorrect_rows = df[df['CORRECT'] == False]

percentage_correct_with_named_entities = (correct_rows['PLACES'].apply(lambda x: bool(x))).mean() * 100
percentage_incorrect_with_named_entities = (incorrect_rows['PLACES'].apply(lambda x: bool(x))).mean() * 100

print(f"The percentage of rows with a GPE/LOC named entity in 'REAL ANSWER': {percentage_with_named_entities:.3f}%")
print(f"The percentage of rows with a GPE/LOC named entity in 'REAL ANSWER' where CORRECT=True: {percentage_correct_with_named_entities:.3f}%")
print(f"The percentage of rows with a GPE/LOC named entity in 'REAL ANSWER' where CORRECT=False: {percentage_incorrect_with_named_entities:.3f}%")


NameError: name 'nlp' is not defined

In [5]:
df = pd.read_csv('no_training_all_question_answers.csv')
df2 = df[df['CLUE CATEGORY'].str.contains('"')] 
df3 = df2.loc[:, ['CLUE CATEGORY', 'CORRECT']]
correct_rows = df3[df3['CORRECT'] == True]
incorrect_rows = df3[df3['CORRECT'] == False]
print("Percentage of correct answers of the model for questions that must contain a specific letter(s) is: ", len(correct_rows)/len(df3))
print("Percentage of incorrect answersof the model for questions that must contain a specific letter(s) is: ", len(incorrect_rows)/len(df3))
print(len(df3))

Percentage of correct answers of the model for questions that must contain a specific letter(s) is:  0.11086309523809523
Percentage of incorrect answersof the model for questions that must contain a specific letter(s) is:  0.8891369047619048
1344


In [49]:
df = pd.read_csv('no_training_all_question_answers.csv')
# df2 = df.loc[:, ['SEASON', 'CLUE VALUE', 'CORRECT']]
df2 = df.loc[:, ['SEASON', 'CLUE VALUE', 'REAL ANSWER']]
level1 = []
level2 = []
level3 = []
level4 = []
level5 = []
finaljeopardy = []
for index, category in df2.iterrows():
    if category[0] <= 16:
        if category[1] == 100:
            level1.append(category[2])
        elif category[1] == 200:
            if index != 0 and index != len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 100 or df2.at[index + 1, 'CLUE VALUE'] == 300:
                    #print("single")
                    level2.append(category[2])
                else:
                    level1.append(category[2])
                    #print('double')
            elif index == 0:
                if df2.at[index + 1, 'CLUE VALUE'] == 300:
                    #print('single')
                    level2.append(category[2])
                else:
                    #print('double')
                    level1.append(category[2])
            elif index == len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 100:
                    level2.append(category[2])
                    #print('single')
                else:
                    level1.append(category[2])
                    #print('double')       
        elif category[1] == 300 or category[1] == 600:
            level3.append(category[2])
        elif category[1] == 400:
            if index != 0 and index != len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 300 or df2.at[index + 1, 'CLUE VALUE'] == 500:
                    #print("single")
                    level4.append(category[2])
                else:
                    #print('double')
                    level2.append(category[2])
            elif index == 0:
                if df2.at[index + 1, 'CLUE VALUE'] == 500:
                    #print("single")
                    level4.append(category[2])
                else:
                    #print('double')
                    level2.append(category[2])
            elif index == len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 300:
                    #print("single")
                    level4.append(category[2])
                else:
                    #print('double')
                    level2.append(category[2])
        elif category[1] == 500 or category[1] == 1000:
            level5.append(category[2])
        elif category[1] == 800:
            level4.append(category[2])
        else:
            finaljeopardy.append(category[2])
    elif category[0] > 16:
        if category[1] == 200:
            level1.append(category[2])
        elif category[1] == 400:
            if index != 0 and index != len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 200 or df2.at[index + 1, 'CLUE VALUE'] == 600:
                    #print("single")
                    level2.append(category[2])
                else:
                    level1.append(category[2])
                    #print('double')
            elif index == 0:
                if df2.at[index + 1, 'CLUE VALUE'] == 600:
                    #print('single')
                    level2.append(category[2])
                else:
                    #print('double')
                    level1.append(category[2])
            elif index == len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 200:
                    level2.append(category[2])
                    #print('single')
                else:
                    level1.append(category[2])
                    #print('double')       
        elif category[1] == 600 or category[1] == 1200:
            level3.append(category[2])
        elif category[1] == 800:
            if index != 0 and index != len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 600 or df2.at[index + 1, 'CLUE VALUE'] == 1000:
                    #print("single")
                    level4.append(category[2])
                else:
                    #print('double')
                    level2.append(category[2])
            elif index == 0:
                if df2.at[index + 1, 'CLUE VALUE'] == 1000:
                    #print("single")
                    level4.append(category[2])
                else:
                    #print('double')
                    level2.append(category[2])

            elif index == len(df2):
                if df2.at[index - 1, 'CLUE VALUE'] == 600:
                    #print("single")
                    level4.append(category[2])
                else:
                    #print('double')
                    level2.append(category[2])
        elif category[1] == 1000 or category[1] == 2000:
            level5.append(category[2])
        elif category[1] == 1600:
            level4.append(category[2])
        else:
            finaljeopardy.append(category[2])
print('Accuracy of model on "level 1" questions:', level1.count(True)/len(level1))
print('Accuracy of model on "level 2" questions:', level2.count(True)/len(level2))
print('Accuracy of model on "level 3" questions:', level3.count(True)/len(level3))
print('Accuracy of model on "level 4" questions:', level4.count(True)/len(level4))
print('Accuracy of model on "level 5" questions:', level5.count(True)/len(level5))
print('Accuracy of model on "final jeopardy" questions:', finaljeopardy.count(True)/len(finaljeopardy))

# NER stuff with levels

df_lvl_1 = pd.DataFrame(level1)
df_lvl_2 = pd.DataFrame(level2)
df_lvl_3 = pd.DataFrame(level3)
df_lvl_4 = pd.DataFrame(level4)
df_lvl_5 = pd.DataFrame(level5)

levels = [df_lvl_1, df_lvl_2, df_lvl_3, df_lvl_4, df_lvl_5]
import spacy

nlp = spacy.load('en_core_web_lg')      # need to run python -m spacy download en_core_web_lg

def extract_named_entities(text):
    doc = nlp(str(text))
    entities = [ent.text for ent in doc.ents if ent.label_ == 'PERSON' or 
                                                ent.label_ == 'NORP' or
                                                ent.label_ == 'FAC' or
                                                ent.label_ == 'ORG' or
                                                ent.label_ == 'GPE' or
                                                ent.label_ == 'LOC' or
                                                ent.label_ == 'PRODUCT' or
                                                ent.label_ == 'EVENT' or
                                                ent.label_ == 'WORK_OF_ART']
    return entities

for i, lvl in enumerate(levels):
    df['named_entities'] = lvl.T.apply(extract_named_entities)

    # print(df['named_entities'].count())
    # print(df['named_entities'].shape[0])

    percentage_with_named_entities = (df['named_entities'].count() / df['named_entities'].shape[0]) * 100

    print(f"level {i}")
    print(f"The percentage of rows with a named entity in 'REAL ANSWER': {percentage_with_named_entities:.3f}%")


Accuracy of model on "level 1" questions: 0.0
Accuracy of model on "level 2" questions: 0.0
Accuracy of model on "level 3" questions: 0.0
Accuracy of model on "level 4" questions: 0.0
Accuracy of model on "level 5" questions: 0.0
Accuracy of model on "final jeopardy" questions: 0.0
level 0
The percentage of rows with a named entity in 'REAL ANSWER': 19.853%
level 1
The percentage of rows with a named entity in 'REAL ANSWER': 19.767%
level 2
The percentage of rows with a named entity in 'REAL ANSWER': 19.865%
level 3
The percentage of rows with a named entity in 'REAL ANSWER': 19.608%
level 4
The percentage of rows with a named entity in 'REAL ANSWER': 19.203%


In [ ]:
# reading in original data from the respective seasons, not necessary but nice to have maybe

df = pd.read_csv('jeopardy_clue_dataset-master/seasons/season34.tsv', sep='\t')
df = pd.read_csv('jeopardy_clue_dataset-master/seasons/season16.tsv', sep='\t')
df = pd.read_csv('jeopardy_clue_dataset-master/combined_season1-39.tsv', sep='\t')
df

,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes
0,1,100,0,LAKES & RIVERS,NaN,River mentioned most often in the Bible,the Jordan,1984-09-10,NaN
1,1,200,0,LAKES & RIVERS,NaN,Scottish word for lake,loch,1984-09-10,NaN
2,1,400,0,LAKES & RIVERS,NaN,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,NaN
3,1,500,0,LAKES & RIVERS,NaN,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,NaN
4,1,100,0,INVENTIONS,NaN,Marconi's wonderful wireless,a radio,1984-09-10,NaN
...,...,...,...,...,...,...,...,...,...
468312,2,400,0,"\""CC"" ME",NaN,"The American Cancer Society says, ""Stay away f...",tobacco,2023-07-28,NaN
468313,2,800,0,"\""CC"" ME",NaN,"In the 5 stages of grief, it comes last",acceptance,2023-07-28,NaN
468314,2,1200,0,"\""CC"" ME",NaN,It begins as a hot dry desert wind over northe...,a sirocco,2023-07-28,NaN
468315,2,2000,0,"\""CC"" ME",NaN,In medicine it's the complete or partial obstr...,occlusion,2023-07-28,NaN
